# Warren 2020 Models

Data from "Constraining properties of the next nearby core-collapse supernova with multi-messenger signals: multi-messenger signals" by Warren, MacKenzie; Couch, Sean; O'Connor, Evan; Morozova, Viktoriya.

1D FLASH simulations with STIR, for alpha_lambda = 1.23, 1.25, and 1.27.  Run with SFHo EOS, M1 with 12 energy groups.

For more information on these simulations, see Warren, Couch, O'Connor, & Morozova ([arXiv:1912.03328](https://arxiv.org/abs/1912.03328)) and Couch, Warren, & O'Connor (2020).

Includes the multi-messenger data from the STIR simulations. The filename indicates the turbulent mixing parameter a and progenitor mass m of the simulation.  Columns are time [s], shock radius [cm], explosion energy [ergs], electron neutrino mean energy [MeV], electron neutrino rms energy [MeV], electron neutrino luminosity [10^51 ergs/s], electron antineutrino mean energy [MeV], electron antineutrino rms energy [MeV], electron antineutrino luminosity [10^51 ergs/s], x neutrino mean energy [MeV], x neutrino rms energy [MeV], x neutrino luminosity [10^51 ergs/s], gravitational wave frequency from eigenmode analysis of the protoneutron star structure [Hz].  Note that the x neutrino luminosity is for one neutrino flavor - to get the total mu/tau neutrino and antineutrino luminosities requires multiplying this number by 4.

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

from astropy import units as u 

from snewpy.neutrino import Flavor, MassHierarchy
from snewpy.models.ccsn import Warren_2020
from snewpy.flavor_transformation import NoTransformation, AdiabaticMSW, ThreeFlavorDecoherence

mpl.rc('font', size=16)
%matplotlib inline

## Initialize Models

To start, let’s see what progenitors are available for the `Warren_2020` model. We can use the `param` property to view all physics parameters and their possible values:

In [ ]:
Warren_2020.param

Quite a lot of choice there! Let’s use the `get_param_combinations` function to get a list of all valid combinations and filter it:

In [ ]:
# This will print a long tuple of combinations:
#Warren_2020.get_param_combinations()

# To get a more manageable list, let’s filter it to show only the
# available progenitors with integer masses and mixing of 1.23:
filtered_models = list(params for params in Warren_2020.get_param_combinations()
                              if params['turbmixing_param'] == 1.23
                              and params['progenitor_mass'].value.is_integer()
                              and 10 <= params['progenitor_mass'].value <= 25)
print("Filtered list of progenitors:\n", *filtered_models, sep='\n')


We’ll pick one of these progenitors and initialise it. If this is the first time you’re using this progenitor, snewpy will automatically download the required data file for you.

In [ ]:
m10 = Warren_2020(**filtered_models[0])
m25 = Warren_2020(**filtered_models[-1])
# This is equivalent to:
#m10 = Warren_2020(progenitor_mass=10*u.solMass, turbmixing_param=1.23)
#m25 = Warren_2020(progenitor_mass=25*u.solMass, turbmixing_param=1.23)

m10

Finally, let’s plot the luminosity of different neutrino flavors for this model. (Note that the `Warren_2020` simulations didn’t distinguish between $\nu_x$ and $\bar{\nu}_x$, so both have the same luminosity.)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5), sharex=True, sharey=True, tight_layout=True)

for i, model in enumerate([m10, m25]):
    ax = axes[i]
    for flavor in Flavor:
        ax.plot(model.time, model.luminosity[flavor]/1e51,  # Report luminosity in units foe/s
                label=flavor.to_tex(),
                color='C0' if flavor.is_electron else 'C1',
                ls='-' if flavor.is_neutrino else ':',
                lw=2)
    ax.set(xlim=(-0.05, 0.65),
           xlabel=r'$t-t_{\rm bounce}$ [s]',
           title=r'{}: {} $M_\odot$'.format(model.metadata['EOS'], model.metadata['Progenitor mass'].value))
    ax.grid()
    ax.legend(loc='upper right', ncol=2, fontsize=18)

axes[0].set(ylabel=r'luminosity [foe s$^{-1}$]');

## Initial and Oscillated Spectra

Plot the neutrino spectra at the source and after the requested flavor transformation has been applied.

### Adiabatic MSW Flavor Transformation: Normal mass ordering

In [ ]:
# Adiabatic MSW effect. NMO is used by default.
xform_nmo = AdiabaticMSW()

# Energy array and time to compute spectra.
# Note that any convenient units can be used and the calculation will remain internally consistent.
E = np.linspace(0,100,201) * u.MeV
t = 50*u.ms

ispec = model.get_initial_spectra(t, E)
ospec_nmo = model.get_transformed_spectra(t, E, xform_nmo)

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(12,5), sharex=True, sharey=True, tight_layout=True)

for i, spec in enumerate([ispec, ospec_nmo]):
    ax = axes[i]
    for flavor in Flavor:
        ax.plot(E, spec[flavor],
                label=flavor.to_tex(),
                color='C0' if flavor.is_electron else 'C1',
                ls='-' if flavor.is_neutrino else ':', lw=2,
                alpha=0.7)

    ax.set(xlabel=r'$E$ [{}]'.format(E.unit),
           title='Initial Spectra: $t = ${:.1f}'.format(t) if i==0 else 'Oscillated Spectra: $t = ${:.1f}'.format(t))
    ax.grid()
    ax.legend(loc='upper right', ncol=2, fontsize=16)

ax = axes[0]
ax.set(ylabel=r'flux [erg$^{-1}$ s$^{-1}$]')

fig.tight_layout();